# Step 2 Dengue Nowcasting (ARX + Joint Loss)

This notebook implements Step 2 according to the requirements:
- Training set: months **<= split_month** (rolling splits)
- Test set: months **> split_month** (for evaluation only)
- Joint loss: WHO monthly supervision + OpenDengue yearly constraints + L2 regularization
- Use Step 1 estimates to impute missing WHO observations as $\tilde{x}$ for lagged inputs
- Output complete parameters, training curves, predictions, and plots

Overfitting control: L2 regularization + early stopping + train/test split.

In [31]:
# If needed, install dependencies in your environment:
# pip install -U pandas numpy torch matplotlib

# Prevent kernel crashes from OpenMP/MKL library conflicts
import os
os.environ.setdefault('KMP_DUPLICATE_LIB_OK', 'TRUE')
os.environ.setdefault('OMP_NUM_THREADS', '1')
os.environ.setdefault('MKL_NUM_THREADS', '1')

import json
from dataclasses import dataclass, asdict
from pathlib import Path
from typing import Tuple, List

import numpy as np
import pandas as pd
import torch
import matplotlib.pyplot as plt

# Further limit thread count
torch.set_num_threads(1)

In [ ]:
@dataclass
class Step2Config:
    # Inputs
    data_path: str = 'master_data.csv'
    step1_predictions_path: str = os.path.join('outputs_step1_wiki', 'predictions_step1_monthly.csv')
    step1_pred_col: str = 'x_pred'

    # Modeling window
    start_month: str = '2021-01'

    # Rolling split evaluation (inclusive)
    rolling_start: str = '2024-11'
    rolling_end: str = '2025-08'
    # To keep rolling evaluation runtime reasonable, you can override training budget per split
    rolling_max_epochs: int = 8000
    rolling_patience: int = 400

    # Target/source names in master_data.csv
    target_source: str = 'WHO'

    # Exogenous signals
    google_sources: Tuple[str, str] = (
        'Google_Trends_Dengue_fever',
        'Google_Trends_Dengue_vaccine',
    )


    # Wikipedia pageviews (external regressor)
    use_wiki: bool = True
    wiki_path: str = 'total_dengue_views.csv'
    wiki_month_col: str = 'Month'
    wiki_value_col: str = 'Total_Views'
    wiki_transform: str = 'log1p'  # 'log1p' or 'none'

    # Feature engineering
    lags_y: Tuple[int, ...] = (1, 2, 12)  # include seasonal lag 12 for monthly data
    use_month_dummies: bool = False  # set False to use Fourier terms instead

    # Seasonality (periodic time series)
    seasonal_period: int = 12  # monthly seasonality
    use_fourier: bool = True  # set True to use Fourier terms instead of month dummies
    fourier_K: int = 2         # number of Fourier harmonics
    # OpenDengue yearly proxy sources (priority)
    yearly_proxy_sources_priority: Tuple[str, ...] = (
        'OpenDengue_State_Aggregated',
        'OpenDengue_National_Yearly',
    )

    # Loss weights - Conservative: balance all three components
    lambda_who: float = 5.0  # Moderate WHO weight
    lambda_year: float = 0.5  # Strong yearly constraint to anchor predictions
    lambda_reg: float = 1e-4  # Base regularization
    lambda_lag_reg: float = 5e-3  # Moderate lag regularization

    # Optimization
    epochs: int = 30000  
    lr: float = 3e-3  # Slower learning rate for stability
    seed: int = 42
    target_scale: float = 1000.0

    # Overfitting controls
    early_stop: bool = True
    patience: int = 1000  # Large patience for stable convergence
    min_delta: float = 1e-7  # Smaller threshold for early stopping

    # Scaling features (standardize using train stats)
    standardize_features: bool = True

    # Output
    outdir: str = 'outputs_step2_period'
    clip_nonnegative: bool = True


cfg = Step2Config()
cfg

Step2Config(data_path='master_data.csv', step1_predictions_path='outputs_step1_wiki/predictions_step1_monthly.csv', step1_pred_col='x_pred', start_month='2021-01', rolling_start='2024-11', rolling_end='2025-08', rolling_max_epochs=8000, rolling_patience=400, target_source='WHO', google_sources=('Google_Trends_Dengue_fever', 'Google_Trends_Dengue_vaccine'), use_wiki=True, wiki_path='total_dengue_views.csv', wiki_month_col='Month', wiki_value_col='Total_Views', wiki_transform='log1p', lags_y=(1, 2, 12), use_month_dummies=False, seasonal_period=12, use_fourier=True, fourier_K=2, yearly_proxy_sources_priority=('OpenDengue_State_Aggregated', 'OpenDengue_National_Yearly'), lambda_who=5.0, lambda_year=0.5, lambda_reg=0.0001, lambda_lag_reg=0.005, epochs=30000, lr=0.003, seed=42, target_scale=1000.0, early_stop=True, patience=1000, min_delta=1e-07, standardize_features=True, outdir='outputs_step2_period', clip_nonnegative=True)

In [33]:
def ensure_outdir(path: str) -> None:
    os.makedirs(path, exist_ok=True)


def parse_monthly_date_any(s: pd.Series) -> pd.Series:
    x = s.astype(str)
    is_ym = x.str.match('^[0-9]{4}-[0-9]{2}$')
    x = np.where(is_ym, x + '-01', x)
    dt = pd.to_datetime(x, errors='coerce')
    return dt


def load_master_csv(path: str) -> pd.DataFrame:
    df = pd.read_csv(path)
    expected = {'resolution', 'date', 'value', 'source'}
    missing = expected - set(df.columns)
    if missing:
        raise ValueError(f"master_data.csv missing columns: {sorted(missing)}")
    return df


def build_monthly_wide(df: pd.DataFrame) -> pd.DataFrame:
    m = df[df['resolution'].astype(str).str.lower().eq('monthly')].copy()
    m['date'] = parse_monthly_date_any(m['date'])
    m = m.dropna(subset=['date']).copy()
    wide = (
        m.pivot_table(index='date', columns='source', values='value', aggfunc='mean')
        .sort_index()
    )
    return wide


def build_yearly_proxy(df: pd.DataFrame, priority_sources: Tuple[str, ...]) -> pd.DataFrame:
    y = df[df['resolution'].astype(str).str.lower().eq('yearly')].copy()
    y['year'] = pd.to_numeric(y['date'], errors='coerce').astype('Int64')
    y = y.dropna(subset=['year'])
    y['year'] = y['year'].astype(int)

    pivot = (
        y.pivot_table(index='year', columns='source', values='value', aggfunc='mean')
        .sort_index()
    )

    chosen = []
    for year, row in pivot.iterrows():
        val = np.nan
        src = None
        for s in priority_sources:
            if s in row.index and pd.notna(row[s]):
                val = float(row[s])
                src = s
                break
        if pd.notna(val):
            chosen.append((year, val, src))

    return pd.DataFrame(chosen, columns=['year', 'od_total', 'od_source'])


def load_step1_predictions(cfg: Step2Config) -> pd.Series:
    if not os.path.exists(cfg.step1_predictions_path):
        return pd.Series(dtype=float)

    s1 = pd.read_csv(cfg.step1_predictions_path)
    if 'date' not in s1.columns:
        raise ValueError(f"Step 1 predictions file missing 'date': {cfg.step1_predictions_path}")

    s1['date'] = parse_monthly_date_any(s1['date'])
    s1 = s1.dropna(subset=['date']).copy()
    s1 = s1.sort_values('date')

    col = cfg.step1_pred_col if cfg.step1_pred_col in s1.columns else None
    if col is None:
        cand = [c for c in s1.columns if 'pred' in c.lower()]
        if not cand:
            cand = [c for c in s1.columns if c != 'date']
        if not cand:
            raise ValueError('No prediction column found in Step 1 predictions file.')
        col = cand[0]

    ser = s1.set_index('date')[col].astype(float)
    return ser



def load_wiki_monthly(cfg: Step2Config) -> pd.Series:
    """
    Load monthly Wikipedia pageviews time series.
    Expected columns: cfg.wiki_month_col (YYYY-MM) and cfg.wiki_value_col (numeric)
    Returns: pd.Series indexed by month (Timestamp at month start).
    """
    if (not getattr(cfg, 'use_wiki', False)) or (not os.path.exists(cfg.wiki_path)):
        return pd.Series(dtype=float)

    w = pd.read_csv(cfg.wiki_path)
    if cfg.wiki_month_col not in w.columns or cfg.wiki_value_col not in w.columns:
        raise ValueError(
            f"Wiki file must contain columns '{cfg.wiki_month_col}' and '{cfg.wiki_value_col}'. Got: {list(w.columns)}"
        )

    w = w.copy()
    w['date'] = parse_monthly_date_any(w[cfg.wiki_month_col])
    w = w.dropna(subset=['date']).sort_values('date')
    s = pd.to_numeric(w[cfg.wiki_value_col], errors='coerce')
    ser = pd.Series(s.values, index=w['date']).groupby(level=0).mean()
    return ser

def add_month_dummies(df: pd.DataFrame) -> pd.DataFrame:
    dt = pd.to_datetime(df.index)
    m = pd.get_dummies(dt.month, prefix='m', drop_first=True)
    m.index = df.index
    return pd.concat([df, m.astype(float)], axis=1)



def add_fourier_features(df: pd.DataFrame, period: int = 12, K: int = 2, prefix: str = 's') -> pd.DataFrame:
    """Add Fourier seasonality terms based on month-of-year."""
    dt = pd.to_datetime(df.index)
    m = dt.month.astype(float)
    out = df.copy()
    for k in range(1, int(K) + 1):
        out[f'{prefix}_sin{k}'] = np.sin(2 * np.pi * k * m / float(period))
        out[f'{prefix}_cos{k}'] = np.cos(2 * np.pi * k * m / float(period))
    return out



def build_design_matrix(wide: pd.DataFrame, cfg: Step2Config) -> tuple[pd.DataFrame, np.ndarray, np.ndarray, np.ndarray, List[str]]:
    """
    Build design matrix.
    Critical fix: lagged variables x_tilde must be scaled by target_scale to match target scale.
    """
    start_dt = pd.to_datetime(cfg.start_month + '-01')
    wide = wide.loc[wide.index >= start_dt].copy()

    target = cfg.target_source
    if target not in wide.columns:
        wide[target] = np.nan

    g1, g2 = cfg.google_sources
    for g in [g1, g2]:
        if g not in wide.columns:
            raise ValueError(f"Missing Google source '{g}' in monthly data.")

    y_who = pd.to_numeric(wide[target], errors='coerce')

    step1_ser = load_step1_predictions(cfg)
    wide['STEP1_est'] = step1_ser.reindex(wide.index)

    # x_tilde: WHO when observed, else Step1 estimate (for lagged inputs)
    x_tilde = y_who.where(y_who.notna(), wide['STEP1_est'])
    
    # Critical fix: scale lagged variables to match target scale
    x_tilde_scaled = x_tilde / cfg.target_scale

    X_df = pd.DataFrame(index=wide.index)
    
    # Normalize Google Trends from [0, 100] to [0, 1]
    X_df['g_fever'] = pd.to_numeric(wide[g1], errors='coerce') / 100.0
    X_df['g_vaccine'] = pd.to_numeric(wide[g2], errors='coerce') / 100.0


    # Wikipedia pageviews regressor
    if getattr(cfg, 'use_wiki', False):
        wiki_ser = load_wiki_monthly(cfg)
        wide['WIKI_raw'] = wiki_ser.reindex(wide.index)
        wiki_raw = pd.to_numeric(wide['WIKI_raw'], errors='coerce')
        wiki_raw = wiki_raw.interpolate(limit_direction='both').ffill().bfill()
        if str(getattr(cfg, 'wiki_transform', 'log1p')).lower() == 'log1p':
            X_df['wiki_views'] = np.log1p(wiki_raw.astype(float))
        else:
            X_df['wiki_views'] = wiki_raw.astype(float)

    # Use scaled values for lagged variables
    for k in cfg.lags_y:
        X_df[f'x_tilde_lag{k}'] = x_tilde_scaled.shift(k)

    # Seasonality features (periodic time series)
    # Prefer Fourier terms when enabled; otherwise fall back to month dummies.
    if getattr(cfg, 'use_fourier', False):
        X_df = add_fourier_features(
            X_df,
            period=int(getattr(cfg, 'seasonal_period', 12)),
            K=int(getattr(cfg, 'fourier_K', 2)),
            prefix='s'
        )
    elif cfg.use_month_dummies:
        X_df = add_month_dummies(X_df)


    X_df['intercept'] = 1.0

    # Light imputation on exogenous features only (lags can remain NaN and be imputed later)
    exo_cols = ['g_fever', 'g_vaccine'] + (['wiki_views'] if 'wiki_views' in X_df.columns else [])
    for col in exo_cols:
        X_df[col] = X_df[col].interpolate(limit_direction='both').ffill().bfill()

    feature_cols = [c for c in X_df.columns if c != 'intercept'] + ['intercept']
    X = X_df[feature_cols].to_numpy(dtype=np.float32)
    y = y_who.to_numpy(dtype=np.float32)
    mask_who = np.isfinite(y)

    return X_df, X, y, mask_who, feature_cols


def build_year_constraints(dates: pd.DatetimeIndex, yearly_proxy: pd.DataFrame, train_mask: np.ndarray) -> List[tuple]:
    df_dates = pd.DataFrame({'date': pd.to_datetime(dates)})
    df_dates['year'] = df_dates['date'].dt.year
    df_dates['month'] = df_dates['date'].dt.month

    constraints = []
    for _, r in yearly_proxy.iterrows():
        y = int(r['year'])
        od_total = float(r['od_total'])
        od_src = str(r['od_source'])

        idx = df_dates.index[df_dates['year'].eq(y)].to_numpy()
        if len(idx) == 0:
            continue

        # require all 12 months and all months in training window
        months_present = set(df_dates.loc[idx, 'month'].tolist())
        if months_present != set(range(1, 13)):
            continue
        if not np.all(train_mask[idx]):
            continue

        constraints.append((y, idx, od_total, od_src))

    return constraints


def time_series_split_mask(index: pd.DatetimeIndex, split_month: str):
    split_dt = pd.to_datetime(split_month + '-01')
    train_mask = index <= split_dt
    test_mask = index > split_dt
    return train_mask, test_mask


def standardize_features(X: np.ndarray, train_mask: np.ndarray, skip_cols: List[int] = None):
    """
    Standardize feature matrix.
    skip_cols: column indices to skip standardization (e.g., intercept term).
    """
    X2 = X.copy()
    n, p = X2.shape
    means = np.zeros(p, dtype=np.float32)
    stds = np.ones(p, dtype=np.float32)
    
    if skip_cols is None:
        skip_cols = []

    for j in range(p):
        col = X2[:, j]
        col_train = col[train_mask]
        
        # For intercept or specified columns, skip standardization
        if j in skip_cols:
            # Only impute NaN
            col = np.where(np.isfinite(col), col, 0.0)
            X2[:, j] = col
            continue
        
        m = np.nanmean(col_train)
        s = np.nanstd(col_train)
        if not np.isfinite(m):
            m = 0.0
        if (not np.isfinite(s)) or s <= 1e-12:
            s = 1.0
        means[j] = m
        stds[j] = s

        # impute NaNs with train mean, then standardize
        col = np.where(np.isfinite(col), col, m)
        X2[:, j] = (col - m) / s

    return X2, means, stds


def train_step2_joint_loss(
    X: np.ndarray,
    y_who: np.ndarray,
    mask_who: np.ndarray,
    year_constraints: List[tuple],
    train_mask: np.ndarray,
    cfg: Step2Config,
    feature_cols: List[str],
    ):
    """
    Train with differentiated regularization: stronger penalty on lag coefficients.
    """
    device = torch.device('cpu')

    X_t = torch.tensor(X, dtype=torch.float32, device=device)
    y_scaled = y_who / cfg.target_scale
    y_t = torch.tensor(y_scaled, dtype=torch.float32, device=device)
    mask_who_t = torch.tensor(mask_who & train_mask, dtype=torch.bool, device=device)

    year_terms = []
    for (year, idx, od_total, od_src) in year_constraints:
        year_terms.append(
            (year, torch.tensor(idx, dtype=torch.long, device=device), float(od_total / cfg.target_scale), od_src)
        )

    # Identify lag feature indices for differentiated regularization
    lag_indices = [i for i, col in enumerate(feature_cols) if 'lag' in col.lower()]

    p = X_t.shape[1]
    beta = torch.nn.Parameter(torch.zeros(p, dtype=torch.float32, device=device))
    opt = torch.optim.Adam([beta], lr=cfg.lr)

    rows = []
    best_loss = np.inf
    best_beta = None
    bad_epochs = 0

    for epoch in range(1, cfg.epochs + 1):
        opt.zero_grad(set_to_none=True)
        x = X_t @ beta

        if mask_who_t.any():
            diff_who = x[mask_who_t] - y_t[mask_who_t]
            L_who = (diff_who ** 2).mean()
        else:
            L_who = torch.tensor(0.0, device=device)

        if len(year_terms) > 0:
            diffs = []
            for _, idx_t, od_total_scaled, _ in year_terms:
                year_sum = x.index_select(0, idx_t).sum()
                diffs.append((year_sum - od_total_scaled) ** 2)
            L_year = torch.stack(diffs).mean()
        else:
            L_year = torch.tensor(0.0, device=device)

        # Differentiated regularization: base + extra penalty on lag coefficients
        L_reg_base = (beta ** 2).sum()
        if lag_indices:
            beta_lag = beta[lag_indices]
            L_reg_lag = (beta_lag ** 2).sum()
        else:
            L_reg_lag = torch.tensor(0.0, device=device)
        
        L_reg = L_reg_base
        L_total = cfg.lambda_who * L_who + cfg.lambda_year * L_year + cfg.lambda_reg * L_reg + cfg.lambda_lag_reg * L_reg_lag
        L_total.backward()
        opt.step()

        val = float(L_total.detach().cpu().item())
        rows.append({
            'epoch': epoch,
            'L_total': val,
            'L_who': float(L_who.detach().cpu().item()),
            'L_year': float(L_year.detach().cpu().item()),
            'L_reg': float(L_reg.detach().cpu().item()),
            'L_lag_reg': float(L_reg_lag.detach().cpu().item()) if lag_indices else 0.0,
        })

        if not np.isfinite(val):
            raise RuntimeError('Training diverged (loss is NaN/Inf).')

        if cfg.early_stop:
            if val < best_loss - cfg.min_delta:
                best_loss = val
                best_beta = beta.detach().cpu().numpy().copy()
                bad_epochs = 0
            else:
                bad_epochs += 1
                if bad_epochs >= cfg.patience:
                    break

    beta_hat = beta.detach().cpu().numpy() if best_beta is None else best_beta
    loss_df = pd.DataFrame(rows)
    return beta_hat, loss_df

In [34]:
def compute_rmse(y_true, y_pred) -> float:
    y_true = np.asarray(y_true, dtype=float)
    y_pred = np.asarray(y_pred, dtype=float)
    return float(np.sqrt(np.nanmean((y_true - y_pred) ** 2)))


def safe_mape(y_true, y_pred) -> float:
    y_true = np.asarray(y_true, dtype=float)
    y_pred = np.asarray(y_pred, dtype=float)
    denom = np.where(np.abs(y_true) < 1e-12, np.nan, np.abs(y_true))
    return float(np.nanmean(np.abs((y_true - y_pred) / denom)) * 100.0)


def make_predictions(X: np.ndarray, beta_hat: np.ndarray, cfg: Step2Config) -> np.ndarray:
    x_scaled = X @ beta_hat
    x = x_scaled * cfg.target_scale
    if cfg.clip_nonnegative:
        x = np.maximum(x, 0.0)
    return x


def plot_loss_curve(loss_df: pd.DataFrame, outdir: Path) -> None:
    fig = plt.figure()
    plt.plot(loss_df['epoch'], loss_df['L_total'], label='Total')
    if (loss_df['L_who'] != 0).any():
        plt.plot(loss_df['epoch'], loss_df['L_who'], label='WHO')
    if (loss_df['L_year'] != 0).any():
        plt.plot(loss_df['epoch'], loss_df['L_year'], label='Yearly')
    plt.yscale('log')
    plt.xlabel('Epoch')
    plt.ylabel('Loss (log scale)')
    plt.title('Training Loss Curve (Step 2)')
    plt.legend()
    plt.tight_layout()
    fig.savefig(outdir / 'loss_curve_step2.png', dpi=200)
    plt.close(fig)


def plot_who_vs_pred(dates: pd.DatetimeIndex, y_who: np.ndarray, x_pred: np.ndarray, train_mask: np.ndarray, outdir: Path, fname: str = 'who_vs_pred_step2.png', title: str = None) -> None:
    fig = plt.figure(figsize=(10, 4))
    plt.plot(dates, x_pred, label='Predicted (Step 2)')
    plt.plot(dates, y_who, label='WHO observed')
    split_dt = dates[train_mask].max() if train_mask.any() else None
    if split_dt is not None:
        plt.axvline(split_dt, linestyle='--', color='gray', label='Train/Test split')
    plt.xlabel('Date')
    plt.ylabel('Monthly dengue cases')
    plt.title(title if title is not None else 'WHO vs Prediction (Step 2)')
    plt.legend()
    plt.tight_layout()
    fig.savefig(outdir / fname, dpi=200)
    plt.close(fig)


def plot_yearly_vs_od(pred_df: pd.DataFrame, yearly_proxy: pd.DataFrame, outdir: Path) -> None:
    pred_year = pred_df.groupby('year', as_index=False)['x_pred'].sum().rename(columns={'x_pred': 'pred_year_total'})
    if yearly_proxy.empty:
        return
    merged = pred_year.merge(yearly_proxy, on='year', how='inner').sort_values('year')

    fig = plt.figure()
    plt.plot(merged['year'], merged['pred_year_total'], marker='o', label='Predicted yearly sum')
    plt.plot(merged['year'], merged['od_total'], marker='o', label='OpenDengue yearly total')
    plt.xlabel('Year')
    plt.ylabel('Total dengue cases (year)')
    plt.title('Yearly Aggregation: Prediction vs OpenDengue (Step 2)')
    plt.legend()
    plt.tight_layout()
    fig.savefig(outdir / 'yearly_vs_opendengue_step2.png', dpi=200)
    plt.close(fig)

    merged.to_csv(outdir / 'yearly_comparison_step2.csv', index=False)


def plot_step1_vs_step2(pred_df: pd.DataFrame, outdir: Path):
    if 'x_step1' not in pred_df.columns:
        return
    fig = plt.figure(figsize=(10, 4))
    dates = pd.to_datetime(pred_df['date'] + '-01')
    plt.plot(dates, pred_df['x_step1'], label='Step 1')
    plt.plot(dates, pred_df['x_pred'], label='Step 2')
    plt.xlabel('Date')
    plt.ylabel('Monthly dengue cases')
    plt.title('Step 1 vs Step 2 Predictions')
    plt.legend()
    plt.tight_layout()
    fig.savefig(outdir / 'step1_vs_step2.png', dpi=200)
    plt.close(fig)

## 1) Load Data, Build Features, and Split Train/Test

- Use Step 1 predictions to impute missing WHO observations for $\tilde{x}$
- Build ARX features: Google Trends + lagged variables + month dummies
- Training/Test split is evaluated over a rolling set of split months.

In [35]:
import copy
from pathlib import Path

df = load_master_csv(cfg.data_path)
wide = build_monthly_wide(df)
yearly_proxy = build_yearly_proxy(df, cfg.yearly_proxy_sources_priority)

X_df, X_raw, y_who, mask_who, feature_cols = build_design_matrix(wide, cfg)

rolling_split_months = pd.period_range(cfg.rolling_start, cfg.rolling_end, freq='M').strftime('%Y-%m').tolist()

print('Total months:', len(X_df))
print('WHO observed months:', int(mask_who.sum()))
print('Feature count:', X_raw.shape[1])
print('Rolling split months:', rolling_split_months)


Total months: 60
WHO observed months: 23
Feature count: 11
Rolling split months: ['2024-11', '2024-12', '2025-01', '2025-02', '2025-03', '2025-04', '2025-05', '2025-06', '2025-07', '2025-08']


## 2) Train Step 2 (Joint Loss + Yearly Constraints + Early Stopping)

- Only training period participates in optimization
- WHO monthly + yearly proxy + L2 regularization
- Early stopping to prevent overfitting

In [36]:
# Rolling train/test split evaluation: for each split_month in [rolling_start, rolling_end]
outdir = Path(cfg.outdir)
roll_dir = outdir / 'rolling_splits'
ensure_outdir(str(roll_dir))

metric_rows = []
coef_rows = []

# Find intercept column index (should not be standardized)
intercept_idx = feature_cols.index('intercept') if 'intercept' in feature_cols else None
skip_cols = [intercept_idx] if intercept_idx is not None else []

for split_month in rolling_split_months:
    train_mask, test_mask = time_series_split_mask(X_df.index, split_month)

    if int(train_mask.sum()) == 0 or int(test_mask.sum()) == 0:
        continue

    # Standardize features (using training set statistics only, skip intercept)
    if cfg.standardize_features:
        X_proc, feat_mean, feat_std = standardize_features(X_raw, train_mask, skip_cols=skip_cols)
    else:
        X_proc = X_raw.copy()
        feat_mean = None
        feat_std = None

    # Year constraints (only full years entirely in training window)
    year_constraints = build_year_constraints(X_df.index, yearly_proxy, train_mask)
    year_years = [int(t[0]) for t in year_constraints]

    # Copy cfg for this split (optionally override training budget for rolling)
    cfg_i = copy.deepcopy(cfg)
    cfg_i.split_month = split_month
    cfg_i.epochs = int(getattr(cfg, 'rolling_max_epochs', cfg.epochs))
    cfg_i.patience = int(getattr(cfg, 'rolling_patience', cfg.patience))

    beta_hat, loss_df = train_step2_joint_loss(
        X=X_proc,
        y_who=y_who,
        mask_who=mask_who,
        year_constraints=year_constraints,
        train_mask=train_mask,
        cfg=cfg_i,
        feature_cols=feature_cols,
    )

    # -------------------------
    # Coefficients per split
    # -------------------------
    beta_hat = np.asarray(beta_hat, dtype=float)

    # Convert to WHO units (because make_predictions multiplies by target_scale)
    coef_y_units_per_stdX = beta_hat * float(cfg_i.target_scale)

    # Convert coefficients back to *raw* feature scale so they are comparable across splits
    # If X_j was standardized as (raw - mean)/std, then:
    #   y = sum_j (coef/std_j) * raw_j  +  (intercept - sum_j (coef/std_j)*mean_j)
    if cfg.standardize_features and (feat_mean is not None) and (feat_std is not None):
        coef_y_units_per_rawX = coef_y_units_per_stdX / feat_std
        if intercept_idx is not None:
            intercept_raw = coef_y_units_per_stdX[intercept_idx] - float(np.dot(coef_y_units_per_rawX, feat_mean))
            coef_y_units_per_rawX = coef_y_units_per_rawX.copy()
            coef_y_units_per_rawX[intercept_idx] = intercept_raw
    else:
        coef_y_units_per_rawX = coef_y_units_per_stdX.copy()

    for j, name in enumerate(feature_cols):
        coef_rows.append({
            'split_month': split_month,
            'feature': name,
            'beta_hat': float(beta_hat[j]),
            'coef_y_units_per_stdX': float(coef_y_units_per_stdX[j]),
            'coef_y_units_per_rawX': float(coef_y_units_per_rawX[j]),
        })

    # Predict for all months
    x_pred = make_predictions(X_proc, beta_hat, cfg_i)

    # Evaluation (only on months with WHO observations)
    mask_train_who = train_mask & mask_who
    mask_test_who = test_mask & mask_who

    rmse_train = compute_rmse(y_who[mask_train_who], x_pred[mask_train_who]) if mask_train_who.any() else np.nan
    mape_train = safe_mape(y_who[mask_train_who], x_pred[mask_train_who]) if mask_train_who.any() else np.nan
    rmse_test = compute_rmse(y_who[mask_test_who], x_pred[mask_test_who]) if mask_test_who.any() else np.nan
    mape_test = safe_mape(y_who[mask_test_who], x_pred[mask_test_who]) if mask_test_who.any() else np.nan

    # Seasonal naive baseline using last year's same month (if lag{seasonal_period} feature exists)
    rmse_test_seasonal_naive = np.nan
    mape_test_seasonal_naive = np.nan
    seasonal_lag_name = f'x_tilde_lag{int(getattr(cfg_i, "seasonal_period", 12))}'
    if seasonal_lag_name in feature_cols:
        j_lag = feature_cols.index(seasonal_lag_name)
        seasonal_naive = X_raw[:, j_lag] * float(cfg_i.target_scale)  # back to WHO units
        rmse_test_seasonal_naive = compute_rmse(y_who[mask_test_who], seasonal_naive[mask_test_who]) if mask_test_who.any() else np.nan
        mape_test_seasonal_naive = safe_mape(y_who[mask_test_who], seasonal_naive[mask_test_who]) if mask_test_who.any() else np.nan


    metric_rows.append({
        'split_month': split_month,
        'n_train_months': int(train_mask.sum()),
        'n_test_months': int(test_mask.sum()),
        'n_train_who': int(mask_train_who.sum()),
        'n_test_who': int(mask_test_who.sum()),
        'n_year_constraints': int(len(year_constraints)),
        'year_constraint_years': ';'.join(map(str, year_years)) if len(year_years) else '',
        'RMSE_train': rmse_train,
        'MAPE_train_%': mape_train,
        'RMSE_test': rmse_test,
        'MAPE_test_%': mape_test,
        'RMSE_test_seasonal_naive': rmse_test_seasonal_naive,
        'MAPE_test_seasonal_naive_%': mape_test_seasonal_naive,
        'epochs_used': int(loss_df['epoch'].max()) if (not loss_df.empty and 'epoch' in loss_df.columns) else np.nan,
    })

    # Plot: prediction vs WHO (with split line)
    fname = f'who_vs_pred_step2_split_{split_month}.png'
    title = f'WHO vs Predicted (Step 2) — split_month={split_month}'
    plot_who_vs_pred(X_df.index, y_who, x_pred, train_mask, roll_dir, fname=fname, title=title)

# -------------------------
# Save metrics + coefficients
# -------------------------
rolling_metrics_df = pd.DataFrame(metric_rows).sort_values('split_month')
coef_long_df = pd.DataFrame(coef_rows)

rolling_metrics_df.to_csv(roll_dir / 'rolling_split_metrics.csv', index=False)
coef_long_df.to_csv(roll_dir / 'rolling_split_coefficients_long.csv', index=False)

# Wide-format coefficient tables (easier to scan in Excel)
coef_raw_wide = coef_long_df.pivot(index='split_month', columns='feature', values='coef_y_units_per_rawX').reset_index()
coef_std_wide = coef_long_df.pivot(index='split_month', columns='feature', values='coef_y_units_per_stdX').reset_index()
coef_raw_wide.to_csv(roll_dir / 'rolling_split_coefficients_raw_wide.csv', index=False)
coef_std_wide.to_csv(roll_dir / 'rolling_split_coefficients_std_wide.csv', index=False)

print('Saved rolling plots to:', roll_dir.resolve())
print('Saved metrics to:', (roll_dir / 'rolling_split_metrics.csv').resolve())
print('Saved coefficients (long) to:', (roll_dir / 'rolling_split_coefficients_long.csv').resolve())
print('Saved coefficients (wide/raw) to:', (roll_dir / 'rolling_split_coefficients_raw_wide.csv').resolve())
print('Saved coefficients (wide/std) to:', (roll_dir / 'rolling_split_coefficients_std_wide.csv').resolve())

rolling_metrics_df


Saved rolling plots to: /Users/cathiesmac/Downloads/Nowcast Dengue in India/outputs_step2_period/rolling_splits
Saved metrics to: /Users/cathiesmac/Downloads/Nowcast Dengue in India/outputs_step2_period/rolling_splits/rolling_split_metrics.csv
Saved coefficients (long) to: /Users/cathiesmac/Downloads/Nowcast Dengue in India/outputs_step2_period/rolling_splits/rolling_split_coefficients_long.csv
Saved coefficients (wide/raw) to: /Users/cathiesmac/Downloads/Nowcast Dengue in India/outputs_step2_period/rolling_splits/rolling_split_coefficients_raw_wide.csv
Saved coefficients (wide/std) to: /Users/cathiesmac/Downloads/Nowcast Dengue in India/outputs_step2_period/rolling_splits/rolling_split_coefficients_std_wide.csv


,split_month,n_train_months,n_test_months,n_train_who,n_test_who,n_year_constraints,year_constraint_years,RMSE_train,MAPE_train_%,RMSE_test,MAPE_test_%,RMSE_test_seasonal_naive,MAPE_test_seasonal_naive_%,epochs_used
0,2024-11,47,13,11,12,3,2021;2022;2023,5798.417185,56.722620,9004.680134,78.251963,13004.736797,63.822491,8000
1,2024-12,48,12,12,11,4,2021;2022;2023;2024,3352.272374,22.152340,4193.599153,47.197417,13582.735466,69.476023,8000
2,2025-01,49,11,13,10,4,2021;2022;2023;2024,3445.289073,26.775218,5122.378979,53.832757,14245.188974,75.617242,8000
3,2025-02,50,10,14,9,4,2021;2022;2023;2024,3435.343522,31.550900,5598.812026,51.460368,15005.998652,79.281025,8000
4,2025-03,51,9,15,8,4,2021;2022;2023;2024,3405.577724,35.243392,6054.602613,47.290468,15899.267121,81.798489,8000
5,2025-04,52,8,16,7,4,2021;2022;2023;2024,3333.246561,39.022926,6457.446141,39.459322,16995.299333,90.087162,8000
6,2025-05,53,7,17,6,4,2021;2022;2023;2024,3298.995423,39.947023,6908.970094,35.308843,18348.386646,98.998839,8000
7,2025-06,54,6,18,5,4,2021;2022;2023;2024,3281.308370,41.086907,6835.082545,24.163525,20094.268611,115.578294,8000
8,2025-07,55,5,19,4,4,2021;2022;2023;2024,3255.302692,40.025636,6821.689446,22.154192,20718.814071,102.313578,8000
9,2025-08,56,4,20,3,4,2021;2022;2023;2024,3224.948687,39.311957,8356.052437,33.330488,21525.254942,94.270769,8000


In [37]:
# Save rolling metrics table
if 'rolling_metrics_df' in globals() and len(rolling_metrics_df) > 0:
    rolling_metrics_df.to_csv(roll_dir / 'rolling_split_metrics.csv', index=False)
    print('Saved rolling plots to:', roll_dir.resolve())
    print('Saved metrics to:', (roll_dir / 'rolling_split_metrics.csv').resolve())
else:
    print('No rolling metrics produced. Check rolling_split_months and data coverage.')


Saved rolling plots to: /Users/cathiesmac/Downloads/Nowcast Dengue in India/outputs_step2_period/rolling_splits
Saved metrics to: /Users/cathiesmac/Downloads/Nowcast Dengue in India/outputs_step2_period/rolling_splits/rolling_split_metrics.csv


## 3) Plot and Save Outputs

Includes:
- Training loss curve
- WHO vs predictions (with train/test split line)
- Yearly aggregation vs OpenDengue
- Step1 vs Step2 comparison
- Parameter and prediction tables

In [38]:
# Optional: keep the original single-split outputs if you still want them.
# Set cfg.split_month to one of the rolling months (or any month) and re-run the original pipeline.
#
# Example:
#   cfg.split_month = '2025-04'
#   train_mask, test_mask = time_series_split_mask(X_df.index, cfg.split_month)
#   ... (then follow the original cells 8-11 logic)
